Load necessary packages

In [2]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn import svm, datasets
import csv
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
from sklearn import metrics
import pandas as pd
import tensorflow as tf
import os

from keras.preprocessing import image
from keras.models import Sequential, Model, load_model
from keras.layers import Dense, Activation, Dropout, Flatten, BatchNormalization, Conv2D, Input, Dense, Dropout, MaxPool2D
from keras.utils import to_categorical
from keras.callbacks import ModelCheckpoint
from keras import callbacks
from keras.callbacks import EarlyStopping
from keras.applications.vgg16 import VGG16
from keras.layers.convolutional import ZeroPadding2D
from keras.optimizers import Adamax
from keras.applications import ResNet50

result = ['Angry', 'Disgust', 'Fear', 'Happy', 'Sad', 'Surprise', 'Neutral']

Using TensorFlow backend.


Download Training Data from Google Drive

In [2]:
!pip install -U -q PyDrive

from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# 1. Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

download = drive.CreateFile({'id': '1wn1WdiMqfaBkHjLGgAGo7mgB2iCCpabk'})  #From shareable link
download.GetContentFile('fer2013.csv')
!ls

adc.json  fer2013.csv  sample_data


Load data-set and splite into training-validation and test dataset

In [11]:
def load_data():
	train_data_X, train_data_Y = [], []
	validation_data_X, validation_data_Y = [], []
	test_data_X, test_data_Y = [], []
	with open("fer2013.csv", 'r') as csvfile:
		datareader = csv.reader(csvfile, delimiter =',')
		headers = next(datareader)
		print(headers) 
		for row in datareader:
			emotion = int(row[0])
			usage = row[2]
			pixels = [int(p) for p in row[1].split()]
			if usage == "Training":
				train_data_X.append(pixels)
				train_data_Y.append(emotion)
			elif usage == "PrivateTest":
				validation_data_X.append(pixels)
				validation_data_Y.append(emotion)
			else:
				test_data_X.append(pixels)
				test_data_Y.append(emotion)
	train_data_X, train_data_Y = np.array(train_data_X)/255.0, np.array(train_data_Y)
	validation_data_X, validation_data_Y = np.array(validation_data_X)/255.0, np.array(validation_data_Y)
	test_data_X, test_data_Y = np.array(test_data_X)/255.0, np.array(test_data_Y)

  
	print("train_data ", train_data_X.shape)
	print("validation_data ", validation_data_X.shape)
	print("test_data ", test_data_X.shape)
	return train_data_X, train_data_Y, validation_data_X, validation_data_Y, test_data_X, test_data_Y

train_data_X, train_data_Y, validation_data_X, validation_data_Y, test_data_X, test_data_Y = load_data()

['emotion', 'pixels', 'Usage']
train_data  (28709, 2304)
validation_data  (3589, 2304)
test_data  (3589, 2304)


VGG-16 model

In [0]:
  
# Reshape array
train_data_X = train_data_X.reshape(28709, 48, 48, 1)
validation_data_X = validation_data_X.reshape(3589, 48, 48, 1)
test_data_X = test_data_X.reshape(3589, 48, 48, 1)

train_data_Y = to_categorical(train_data_Y, num_classes=7)
validation_data_Y = to_categorical(validation_data_Y, num_classes=7)
test_data_Y = to_categorical(test_data_Y, num_classes=7)

print("train_data ", train_data_X.shape)
print("train_data ", train_data_Y.shape)
print("validation_data ", validation_data_X.shape)
print("validation_data ", validation_data_Y.shape)
print("test_data ", test_data_X.shape)
print("test_data ", test_data_Y.shape)

# keras.layers.Conv2D(filters, kernel_size, strides=(1, 1), padding='valid', data_format=None, dilation_rate=(1, 1), activation=None, use_bias=True, kernel_initializer='glorot_uniform', bias_initializer='zeros', kernel_regularizer=None, bias_regularizer=None, activity_regularizer=None, kernel_constraint=None, bias_constraint=None)
model = Sequential()

model.add(ZeroPadding2D((1,1), input_shape=(48, 48, 1)))
model.add(Conv2D(64, (3, 3), strides=(1, 1), padding='same', activation='relu'))
#model.add(ZeroPadding2D((1,1)))
model.add(BatchNormalization())
model.add(Conv2D(64, (3, 3), strides=(1, 1), padding='same', activation='relu'))
model.add(MaxPool2D(pool_size=(2, 2), strides=2))

#model.add(ZeroPadding2D((1,1)))
model.add(Conv2D(128, (3, 3), strides=(1, 1), padding='same', activation='relu'))
#model.add(ZeroPadding2D((1,1)))
model.add(BatchNormalization())
model.add(Conv2D(128, (3, 3), strides=(1, 1), padding='same', activation='relu'))
model.add(MaxPool2D(pool_size=(2, 2), strides=2))

#model.add(ZeroPadding2D((1,1)))
model.add(Conv2D(256, (3, 3), strides=(1, 1), padding='same', activation='relu'))
model.add(BatchNormalization())
#model.add(ZeroPadding2D((1,1)))
model.add(Conv2D(256, (3, 3), strides=(1, 1), padding='same', activation='relu'))
model.add(BatchNormalization())
#model.add(ZeroPadding2D((1,1)))
model.add(Conv2D(256, (3, 3), strides=(1, 1), padding='same', activation='relu'))
model.add(MaxPool2D(pool_size=(2, 2), strides=2))

#model.add(ZeroPadding2D((1,1)))
model.add(Conv2D(512, (3, 3), strides=(1, 1), padding='same', activation='relu'))
model.add(BatchNormalization())
#model.add(ZeroPadding2D((1,1)))
model.add(Conv2D(512, (3, 3), strides=(1, 1), padding='same', activation='relu'))
model.add(BatchNormalization())
#model.add(ZeroPadding2D((1,1)))
model.add(Conv2D(512, (3, 3), strides=(1, 1), padding='same', activation='relu'))
model.add(MaxPool2D(pool_size=(2, 2), strides=2))

#model.add(ZeroPadding2D((1,1)))
model.add(Conv2D(512, (3, 3), strides=(1, 1), padding='same', activation='relu'))
model.add(BatchNormalization())
#model.add(ZeroPadding2D((1,1)))
model.add(Conv2D(512, (3, 3), strides=(1, 1), padding='same', activation='relu'))
model.add(BatchNormalization())
#model.add(ZeroPadding2D((1,1)))
model.add(Conv2D(512, (3, 3), strides=(1, 1), padding='same', activation='relu'))
model.add(MaxPool2D(pool_size=(2, 2), strides=2))

# FC layers
model.add(Flatten())
model.add(Dense(4096, activation='relu'))
model.add(Dropout(0.25))
model.add(Dense(4096, activation='relu'))
model.add(Dropout(0.25))
model.add(Dense(1000, activation='relu'))
model.add(Dropout(0.25))
model.add(Dense(7, activation='softmax'))

model.summary()

# compile the model
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# start training the model
hist = model.fit(train_data_X, train_data_Y, batch_size=128, epochs=20, verbose=1, validation_data=(validation_data_X, validation_data_Y), shuffle=True)

# save the current model and weights
#model.save_weights('weights.h5')
model.save('model.h5')

loss, acc = model.evaluate(test_data_X, test_data_Y, batch_size=64)

print("Accuracy: ", acc)
print("Loss: ",loss)


Train on 28709 samples, validate on 3589 samples
Epoch 1/25
28709/28709 [==============================] - 76s 3ms/step - loss: 1.8285 - acc: 0.2639 - val_loss: 1.8375 - val_acc: 0.2497  
Epoch 2/25
28709/28709 [==============================] - 70s 2ms/step - loss: 1.5331 - acc: 0.3842 - val_loss: 1.5009 - val_acc: 0.4452  

Epoch 24/25
28709/28709 [==============================] - 70s 2ms/step - loss: 0.1793 - acc: 0.9454 - val_loss: 1.7357 - val_acc: 0.6266   
Epoch 25/25
28709/28709 [==============================] - 70s 2ms/step - loss: 0.1596 - acc: 0.9510 - val_loss: 2.0783 - val_acc: 0.6141  




  
Test:  
Accuracy:  0.606018389531848
Loss:  2.216669141166753

In [0]:
# save the current model and weights
#model.save_weights('weights.h5')
model.save('model.h5')

loss, acc = model.evaluate(test_data_X, test_data_Y, batch_size=64)

print("Accuracy: ", acc)
print("Loss: ",loss)

!ls
def load_image(img_path, show=True):

    img = image.load_img(img_path, target_size=(48, 48), color_mode = "grayscale")
    img_tensor = image.img_to_array(img)                    # (height, width, channels)
    img_tensor = np.expand_dims(img_tensor, axis=0)         # (1, height, width, channels), add a dimension because the model expects this shape: (batch_size, height, width, channels)
    img_tensor /= 255.
    #print(img_tensor.shape)
    if show:
        plt.imshow(img, cmap='gray')

    return img_tensor
    # show the image


if __name__ == "__main__":

    # load model
    #model = load_model("model.h5")

    # image path
    !wget https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcTyndHmil5Mdk8-v-6To_6ncm-l8-L5n0l0Bh5dcOR6ZUkq-pODmQ #Angry
    !wget https://www.sciencedaily.com/images/2009/03/090312093916_1_540x360.jpg #Disgust
    !wget https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcREqmMQoG0_0XOYf5YWhLIdo-xZmohGMOWm0P4aFUHpFzYiB_yY #Fear
    !wget https://w3.chabad.org/media/images/689/tyZo6894268.jpg #Happy
    !wget https://c1.staticflickr.com/4/3675/9436653177_fd00cc9d2c_b.jpg #Sad
    !wget https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcTt54l_3Sud9ektJH69BXBOADn3qk7d0XrTpjrAWJ6u5omaX2NlcQ # Surprise 
    !wget https://goldenmeancalipers.com/wp-content/uploads/2011/12/mirror11.jpg # Netural

    # load a single image
    new_image = load_image('mirror11.jpg')
    
    # check prediction
    pred = model.predict(new_image)
    index = pred.argmax()
    print(result[index])
    #print(max(pred))
